In [35]:
import networkx as nx
# https://networkx.github.io/documentation/stable/reference/index.html
import matplotlib.pyplot as plt
from matplotlib import pylab
import numpy as np
import pandas as pd

## Dataset 

This dataset comes from a consulting firm. Emails exchanged during the period 4 to 19 March 2019. 1174928 lines

Each line describes what a collaborator sent or received at MessDate. S/he sent (Id_Direction is equal to 1) or received an email from a contact (Id_Direction is equal to 2).

The interlocutor can be interne/externe/unidentified (PartnerTypeName).

The interaction involving the collaborator is defined by:

- GroupName1 : Post/Title of the collaborator

- RecipientName : The collaborator

- MessSize : message size in Mégabytes

- MessDate : Date+Hour

- Id_Direction : 1 for a sent email, 2 for a received email

- PartnerTypeName : the interlocutor may be either:      

 -        Interne
 -        Internet (external interlocutor)
 -        Unidentified local address (applications or server or cloud mail address)

- PartnerName : domain name if external interlocutor

- Recipient_Display : the interlocutor





In [36]:
df = pd.read_csv('dataset_UE_GTSNA.csv', sep=',')
df.head()

/home/cecile/.pyenv/versions/3.8.12/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,GroupName1,MessSize,MessDate,Subject,Id_Direction,PartnerTypeName,Id_Recipient,Id_Regroup,Partner_Name,Recipient_Display,Recipient_Name
0,Senior,"0,00990676879882813",04/03/2019 00:00:02,0de420e8cb4f25a853b2d42c27787d8691176975,2,Internal,852,6674,6d2830b1e76dc7300fce6745176601827d233de8,0d0128c6ca8c4006d5c1d8e1694b3302380755ab,b4b419cd95026fd7f74cc90754ec02f40de14d85
1,Senior,"0,0100250244140625",04/03/2019 00:00:02,43cbcc2246e28d855c636fae99bbd0dd828ac276,2,Internal,852,6674,6d2830b1e76dc7300fce6745176601827d233de8,0d0128c6ca8c4006d5c1d8e1694b3302380755ab,b4b419cd95026fd7f74cc90754ec02f40de14d85
2,Manager,"0,0445833206176758",04/03/2019 00:00:08,a60173eae76ea9bc97aa2f1d84bf78797c77a1f5,2,Internet,1113,593252,3793d3935a39a8038a8a8b27e00c3aa587395412,77c570261748c351a2296ac47685450587bc7d2b,bec1a496d58aefeb626da2e7484e38657704fa19
3,Partner,"0,0256938934326172",04/03/2019 00:00:13,c0eee27f5bb484585a79c5d9a0646097937c0091,2,Unidentified local address,1888,552279,97db2e51f36c0b207c8f6a4ecb488473238d04ae,4d48a3734ac25c0f71584f79716881f9529167f6,253b86e279f1ca0520ed2d4aeffa491ce0054a1c
4,Partner,"0,0256862640380859",04/03/2019 00:00:13,c0eee27f5bb484585a79c5d9a0646097937c0091,2,Unidentified local address,1888,552279,97db2e51f36c0b207c8f6a4ecb488473238d04ae,4d48a3734ac25c0f71584f79716881f9529167f6,253b86e279f1ca0520ed2d4aeffa491ce0054a1c


In [37]:
df=df.sort_values(by = ['MessDate','Subject'])

In [38]:
#df.head(100000).to_csv("mails_conseil_100000_sorted_MessDate.csv", header=True, index=False)

In [39]:
# how many Nan values?
df['GroupName1'].isna().sum()

0

In [40]:
df.describe()

,Id_Direction,Id_Recipient,Id_Regroup
count,1.174928e+06,1.174928e+06,1.174928e+06
mean,1.630459e+00,2.215358e+05,4.795618e+05
std,4.826807e-01,5.005959e+05,5.783001e+05
min,1.000000e+00,6.800000e+02,1.320000e+02
25%,1.000000e+00,1.248000e+03,1.733000e+03
50%,2.000000e+00,1.653000e+03,2.011780e+05
75%,2.000000e+00,1.342900e+04,7.340110e+05
max,2.000000e+00,1.896509e+06,1.906385e+06


1.17 million de lines

In [42]:
df[['Recipient_Name','MessDate','Recipient_Display','Subject']].groupby(["Subject",'MessDate'], as_index=False).count()

,Subject,MessDate,Recipient_Name,Recipient_Display
0,00002ab407c3e60791d1ab3a4c85acfa031248fd,08/03/2019 19:34:37,2,2
1,00002ab407c3e60791d1ab3a4c85acfa031248fd,09/03/2019 16:52:42,3,3
2,00002ab407c3e60791d1ab3a4c85acfa031248fd,09/03/2019 16:52:43,1,1
3,00002f551689bad3b6b2ad809e7a5d49a9bd1d2b,04/03/2019 20:04:26,1,1
4,00002f551689bad3b6b2ad809e7a5d49a9bd1d2b,04/03/2019 20:04:32,1,1
...,...,...,...,...
509949,fffd9335dc8b884714a54dd361376e5e1f106d80,05/03/2019 15:10:22,1,1
509950,fffde77881233f89815a8c85b73ecac7440b6ded,04/03/2019 10:10:54,1,1
509951,fffdffe8684953ba5aa867485187b2a8992d3558,13/03/2019 18:53:13,1,1
509952,fffdffe8684953ba5aa867485187b2a8992d3558,13/03/2019 19:11:24,1,1


509954 messages (unique pairs ["Subject",'MessDate']) in the dataset. 

An email can be described by several lines.

Example1: if A and B are two internal colleagues, there will be 1 line dedicated to A and 1 line dedicated to B for the same message 
~~~
Manager | 04/03/2019 00:03 | 1 | B | A | 4bb26edab1c7a7bd212a86e4308d128af11e117c
Senior  | 04/03/2019 00:03 | 2 | A | B | 4bb26edab1c7a7bd212a86e4308d128af11e117c

~~~

Example2: if A, B, C are 3 colleagues and they all receive the same message from an external partner, there will be 3 lines with same s(ubject,date) with id_Direction = 2.


## Roles

In [28]:
df['GroupName1'].unique()

array(['Senior', 'Manager', 'Partner', 'Staff', 'Senior Manager',
       'Director', 'Junior Staff', 'Title:[no value]',
       'Administrative Staff', 'Assistant Manager'], dtype=object)

In [29]:
df1 = pd.DataFrame({'A': ['a', 'b', 'a', 'a', 'b'], 'B': [8, 4, 5, 10, 8], 'C': ['x', 'x', 'y', 'y', 'x'], 'D': [0, 1, 2, 3, 4]})

In [30]:
df1[['A','B']].groupby('A').count()

,B
A,
a,3
b,2


In [31]:
df1[['A','C']].drop_duplicates().groupby('A').size()

A
a    2
b    1
dtype: int64

In [32]:
# Number of lines per role
df[['GroupName1','Id_Recipient']].groupby('GroupName1').size()

GroupName1
Administrative Staff     51942
Assistant Manager         9009
Director                 81180
Junior Staff             21187
Manager                 232904
Partner                 120964
Senior                  206667
Senior Manager          121972
Staff                   322873
Title:[no value]          6230
dtype: int64

In [33]:
# Number of people per role
df[['GroupName1','Id_Recipient']].drop_duplicates().groupby('GroupName1').size()

GroupName1
Administrative Staff     30
Assistant Manager         4
Director                 65
Junior Staff            116
Manager                 200
Partner                  64
Senior                  316
Senior Manager          114
Staff                   647
Title:[no value]         51
dtype: int64

## Interactions

In [34]:
# Nb of interactions with different people per collaborator
df[["Recipient_Display","Recipient_Name"]].groupby("Recipient_Name").describe()

Recipient_Display         \
                                                     count unique   
Recipient_Name                                                      
0000ef79b21a7e73725841cb8355361fced8457e                 1      1   
000127ddacef5499dc311db7cdcdc41477a2c749                 5      3   
0002a573dcdbda5e716d26500c33c999c5c0e6af                 8      5   
0003192c3e74f2e107f9815c6e00eec9db6fd524                 1      1   
0004309fe945da9a82b18c0a4df07747fed36c7c                 2      2   
...                                                    ...    ...   
fffbcd57cb07cabbd6387362b2d77ab2571b7536                 3      1   
fffbda0c832a0ff89a203d394d53193d51ec0bea                 1      1   
fffc593847ec88edd63d87dc855a41df890c7fe0                 3      1   
fffca35584436327634906d4563b6f0283a3fbd7                 2      2   
ffff619b6c198938d740670ab118fba5483acd95                 4      2   

                                                                                    \
                                                                               top   
Recipient_Name                                                                       
0000ef79b21a7e73725841cb8355361fced8457e  f01d7ddbbe3ae90e7223c67c35fdb03158e2da3f   
000127ddacef5499dc311db7cdcdc41477a2c749  8779838af0b40469a4c4575300b3d7c29c92ae39   
0002a573dcdbda5e716d26500c33c999c5c0e6af  2bc2dcc73b24505cf8cdec15e60bff2d536fc418   
0003192c3e74f2e107f9815c6e00eec9db6fd524  7f0bbc4bc9cdbe6a59b8feb1a99deb61da3dfd02   
0004309fe945da9a82b18c0a4df07747fed36c7c  879a6ab23c9774e4f868ec22ab41071525a20382   
...                                                                            ...   
fffbcd57cb07cabbd6387362b2d77ab2571b7536  b07d1bb755cbb28617579fb4b9923edad4754bd0   
fffbda0c832a0ff89a203d394d53193d51ec0bea  7bce578fe866857b379c210a0bca4bdf0e1a3e9c   
fffc593847ec88edd63d87dc855a41df890c7fe0  df32d3d2ca3b3ebdc3edea9a917d3db4b961ff6f   
fffca35584436327634906d4563b6f0283a3fbd7  a18862457958b9bfc705c1315afc4e143af7fb49   
ffff619b6c198938d740670ab118fba5483acd95  2b5b6fb7400a707765e0090b09fc0a48125d50bc   

                                               
                                         freq  
Recipient_Name                                 
0000ef79b21a7e73725841cb8355361fced8457e    1  
000127ddacef5499dc311db7cdcdc41477a2c749    3  
0002a573dcdbda5e716d26500c33c999c5c0e6af    3  
0003192c3e74f2e107f9815c6e00eec9db6fd524    1  
0004309fe945da9a82b18c0a4df07747fed36c7c    1  
...                                       ...  
fffbcd57cb07cabbd6387362b2d77ab2571b7536    3  
fffbda0c832a0ff89a203d394d53193d51ec0bea    1  
fffc593847ec88edd63d87dc855a41df890c7fe0    3  
fffca35584436327634906d4563b6f0283a3fbd7    1  
ffff619b6c198938d740670ab118fba5483acd95    3  

[46928 rows x 4 columns]